In [5]:
#%%
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy.spatial import KDTree
from sklearn.cluster import DBSCAN
from mpl_toolkits.mplot3d import Axes3D

#%% utility functions
def show_cloud(points_plt):
    ax = plt.axes(projection='3d')
    ax.scatter(points_plt[:,0], points_plt[:,1], points_plt[:,2], s=0.01)
    plt.show()

def show_scatter(x,y):
    plt.scatter(x, y)
    plt.show()

def get_ground_level(pcd, dataset_name="dataset"):
    z_values = pcd[:, 2]  # ta höjddata

    # skapa histogram
    counts, bins = np.histogram(z_values, bins=100)
    max_bin_index = np.argmax(counts)
    ground_level = (bins[max_bin_index] + bins[max_bin_index + 1]) / 2

    # plotta och spara histogram
    plt.figure()
    plt.hist(z_values, bins=100)
    plt.axvline(ground_level, color='red', linestyle='--',
                label=f'Ground level: {ground_level:.2f}')
    plt.legend()
    plt.xlabel("Höjd (Z)")
    plt.ylabel("Antal punkter")
    plt.title(f"Histogram av höjdfördelning ({dataset_name})")
    plt.savefig(f"images/histogram_{dataset_name}.png")
    plt.close()

    return ground_level

#%% Lista med dataset-filer
datasets = ["dataset1.npy", "dataset2.npy"]

#%% Kör analysen för varje dataset
for filename in datasets:
    print(f"Processing {filename}...")
    pcd = np.load(filename)
    
    # beräkna marknivå
    est_ground_level = get_ground_level(pcd, dataset_name=filename.split('.')[0])
    print(f"{filename}: Beräknad marknivå = {est_ground_level:.2f}")
    
    # ta bort marknivå för visualisering
    pcd_above_ground = pcd[pcd[:,2] > est_ground_level]
    
    print(f"{filename}: Antal punkter ovanför marknivå = {pcd_above_ground.shape[0]}")
    
    # visa punktskyen
    %matplotlib qt
    show_cloud(pcd_above_ground)

    # Exempel på DBSCAN-klustring för det första datasetet
    if filename == "dataset1.npy":
        unoptimal_eps = 10
        clustering = DBSCAN(eps=unoptimal_eps, min_samples=5).fit(pcd_above_ground)
        
        clusters = len(set(clustering.labels_)) - (1 if -1 in clustering.labels_ else 0)
        colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, clusters)]
        
        plt.figure(figsize=(10,10))
        plt.scatter(pcd_above_ground[:,0], 
                    pcd_above_ground[:,1],
                    c=clustering.labels_,
                    cmap=matplotlib.colors.ListedColormap(colors),
                    s=2)
        plt.title('DBSCAN: %d clusters' % clusters, fontsize=20)
        plt.xlabel('x axis', fontsize=14)
        plt.ylabel('y axis', fontsize=14)
        plt.show()


Processing dataset1.npy...
dataset1.npy: Beräknad marknivå = 61.25
dataset1.npy: Antal punkter ovanför marknivå = 62411
Processing dataset2.npy...
dataset2.npy: Beräknad marknivå = 61.27
dataset2.npy: Antal punkter ovanför marknivå = 71858


In [6]:
#%%
from sklearn.neighbors import NearestNeighbors

def find_optimal_eps(pcd_above_ground, dataset_name="dataset"):
    # Beräkna k-distans (k = min_samples för DBSCAN)
    min_samples = 5
    neigh = NearestNeighbors(n_neighbors=min_samples)
    neigh.fit(pcd_above_ground[:, :2])  # Endast x och y för avståndsberäkning
    distances, indices = neigh.kneighbors(pcd_above_ground[:, :2])
    
    # Sortera k-distans för elbow-plot
    k_distances = np.sort(distances[:, -1])
    
    # Rita k-distans plot
    plt.figure(figsize=(8,5))
    plt.plot(k_distances)
    plt.xlabel("Points sorted by distance")
    plt.ylabel(f"{min_samples}-NN distance")
    plt.title(f"Elbow plot för DBSCAN eps ({dataset_name})")
    plt.grid(True)
    plt.savefig(f"images/elbow_{dataset_name}.png")
    plt.show()
    
    # Enkel heuristik: välj eps vid knäet som t.ex. median eller 90:e percentil
    optimal_eps = np.percentile(k_distances, 90)
    print(f"{dataset_name}: Optimal eps ~ {optimal_eps:.2f}")
    
    return optimal_eps


#%% Kör DBSCAN med optimalt eps för varje dataset
for filename in datasets:
    print(f"Processing {filename} for Task 2...")
    pcd = np.load(filename)
    est_ground_level = get_ground_level(pcd, dataset_name=filename.split('.')[0])
    pcd_above_ground = pcd[pcd[:,2] > est_ground_level]
    
    # hitta optimal eps
    optimal_eps = find_optimal_eps(pcd_above_ground, dataset_name=filename.split('.')[0])
    
    # applicera DBSCAN med optimal eps
    clustering = DBSCAN(eps=optimal_eps, min_samples=5).fit(pcd_above_ground[:, :2])
    clusters = len(set(clustering.labels_)) - (1 if -1 in clustering.labels_ else 0)
    colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, clusters)]
    
    plt.figure(figsize=(10,10))
    plt.scatter(pcd_above_ground[:,0], 
                pcd_above_ground[:,1],
                c=clustering.labels_,
                cmap=matplotlib.colors.ListedColormap(colors),
                s=2)
    plt.title(f'DBSCAN ({filename}): {clusters} clusters', fontsize=20)
    plt.xlabel('x axis', fontsize=14)
    plt.ylabel('y axis', fontsize=14)
    plt.show()


Processing dataset1.npy for Task 2...
dataset1: Optimal eps ~ 0.30
Processing dataset2.npy for Task 2...
dataset2: Optimal eps ~ 0.27
